# Retail Demo Store Messaging Workshop - Braze

In this workshop we will use [Braze](https://braze.com/) to add the ability to personalize marketing messages to customers of the Retail Demo Store using customer behavioral data and the Personalize models you trained in the prior workshops. We are going to define a campaign in Braze to send target users an email with product recommendations from the Amazon Personalize campaign we created in the Personalization workshop earlier.  You will also create an email template using the Braze Connected Content feature.  When the campaign is launched, Connected Content will send emails to the users that belong in the target group.  These emails will be formatted using the Personalize Campaign you created earlier.

Recommended Time: 1 hour

## Prerequisites

This module uses Amazon Personalize to generate and associate personalized product recommendations for users.  The content of this workshop is presented with the assumption that you have either completed the [Personalization](../1-Personalization/1.1-Personalize.ipynb) workshop or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

This part of the workshop will also require access to a Braze account.  If you are unsure of how to set that up contact <TODO: INSERT CONTACT HERE>.

## Architecture

Before you set up Braze to send personalized messages to users, let's review the relevant parts of the Retail Demo Store architecture and how it uses Braze to integrate with the machine learning campaigns created in Personalize.

![Braze Personalization Architecture](images/braze-personalize-arch.png)

Braze will send emails to your users based on their behavior or based on attributes of their user profiles.  A discussion of Braze campaigns is beyond the scope of this document but Braze will ingest real-time events from mobile and web applications.  This data can be used to identify users and to build user profiles that can be used to determine when to message or email users.  

This event data flow will happen in parallel to the same behavioral event data being sent to Amazon Personalize. In this workshop, the demo store uses Amplify to send events to Personalize as shown in the diagram. This data is used to train a recommendations model that can in turn be used by Braze Connected Content to personalize content to users of your mobile and web applications when your Braze campaign runs.  

Braze Connected content will be able to get these recommendations via a recommendation service running in AWS.  Earlier in this workshop, this service was deployed using by the Cloud Formation templates you used to deploy the workshop environment in ECS.

### Building a Braze Email Template

Before you can run a campaign in Braze, you will need to create an email template that uses Braze Connected Content to fetch Product recommendations for the users you will be emailing via the campaign.

1. Log in to the Braze [console](https://dashboard.braze.com/auth).  You will need to have a Braze account configured for you to continue this workshop, or use your existing Braze account.

![Braze Login Screen](images/braze-create-template-0.png)

1. On the left hand pane of the Braze console, click the Templates & Media link.  This will show the Templates list screen.  Click the From File button under Basic Email Templates at the top of the screen.

![](images/braze-create-template-1.png)

1. A sample HTML template file has been provided for you in this workshop, to save editing time.  You can edit this template as needed but it is recommended that you follow this workshop at least once to faimilarize yourself with how this integration will work.  Click the Upload from File button:

![](images/braze-create-template-2.png)

4. Navigate to your local clone of this repository, to the `/workshop/4-Messaging/` folder.  Select the `braze-connected-content-email-template.zip` file and click the Open or Save button to upload the ZIP file to Braze

![](images/braze-create-template-3.png)

5. Braze will validate the file, and return a status window.  Click the Build Email button to continue

![](images/braze-create-template-4.png)

6. On the next screen, give the template a name, such as `Braze Connected Content Workshop Template`

![](images/braze-create-template-5.png)

7. On the bottom right of the screen, click the Save Template button.  Make sure to click the Save button often, the editor will not auto-save your changes.

![](images/braze-create-template-6.png)

8. Scroll back up and click the Edit Email Body link

![](images/braze-create-template-7.png)

9. This will take you to the HTML template editor.  At the very top, there is a Connected Content call that calls the Personalize recommendation service you deployed eariler, and gets a set of recommendations for the user ID that this template will be sent to:

```
{% connected_content <URL OF YOUR RECOMMENDATION SERVICE>/recommendations?userID={{${user_id}}}&fullyQualifyImageUrls=1&numResults=4 :save result %}
```

You will need to look up the URL for the deployment of your Personalize catalog recommendation service.  Once you have that, you will need to paste the following code into the section above:

```
http://<YOUR RECOMMENDATION SERVICE URL>/recommendations?userID={{${user_id}}}&fullyQualifyImageUrls=1&numResults=4
```

Do not change the URL query parameters; this will break the template in the next few steps.

This snippet will be processed by Braze when you run your campaign, and Connected Content will query your Personalize recommendation service with the `user_id` this email is intended for, fetch recommendations, and then will fill in the appropriate sections of the HTML template with information about recommended products.

![](images/braze-create-template-8.png)

10. Note that this template is hard-coded to show 4 product recommendations.  Once Connected Content loads recommendations, they are available in the `result` array and they can be referenced in the HTML for your template.

When you are finished modifying your template, click the Done button at the bottom right of the editor screen.

![](images/braze-create-template-9.png)

11. Before navigating away from the template editor, make sure to click the Save Template button on the bottom right of the template window, or your earlier changes will be lost.

![](images/braze-create-template-10.png)


### Verify Machine Learning Model / Recommender

If you open a web browser window/tab and browse to the Pinpoint service in the AWS console for the AWS account we're working with, you should see the ML Model / Recommender that we just created in Pinpoint.

![Pinpoint ML Model / Recommender](images/pinpoint-ml-model.png)

### Upload Braze User Profiles

So that we can send an email to more than one endpoint in this workshop, verify a couple more variations of your email address.

Assuming your **valid** email address is `joe@example.com`, add a few more variations using `+` notation such as...

- `joe+1@example.com`
- `joe+2@example.com`
- `joe+3@example.com`

Just enter a variation, click the "Verify email address" button, and repeat until you've added a few more. Write down or commit to memory the variations you created--we'll need them later.

By adding these variations, we're able to create separate Retail Demo Store accounts for each email address and therefore separate endpoints in Pinpoint that we can target. Note that emails sent to the these variations should still be delivered to your same inbox.

### Create a Braze Campaign



## Workshop Complete

Congratulations! You have completed the Retail Demo Store Braze Workshop.

### Cleanup

If you launched the Retail Demo Store in your personal AWS account **AND** you're done with all workshops & your evaluation of the Retail Demo Store, you can remove all provisioned AWS resources and data by deleting the CloudFormation stack you used to deploy the Retail Demo Store.

> If you are participating in an AWS managed event such as a workshop and using an AWS provided temporary account, you can skip the following cleanup steps unless otherwise instructed.